In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import numpy as np

In [ ]:
df_columns = ['team', 'win%', 'date', 'result']
df = pd.DataFrame(columns=df_columns)

output_file = 'z:\python_projects\aaa.exe'
year = 2019
result = ''
URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
source = requests.get(URL, headers=headers)
soup = BeautifulSoup(source.content, 'html.parser')

# Finding year of predictions
for timestamp in soup.findAll('div', attrs={'class': 'container'}):
    for year in timestamp.findAll('div', attrs={'id': 'intro'}):
        year = int(year.h1.get_text()[0:5])
        
# Gathering data
for week in soup.findAll('section', attrs={'class': 'week'}):
    if week.get_text()[5] == '5':
        for date in week.findAll('div', attrs={'class': 'days'}):
            for day in date.findAll('div', attrs={'class': 'day'}):  
                for game in day.findAll('div', attrs={'class': 'game'}):

                    # Finding date of each game  
                    for h4 in day.findAll('h4', attrs={'class': 'h4'}):
                        date = h4.get_text()
                        date = date.split(', ')[1]
                        date = date[0:3] + date[4:6] + ' ' + str(year)
                        date = datetime.strptime(date, '%b %d %Y')
                        date = date.strftime('%m/%d/%Y')


                    # Finding data for each game
                    for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                        for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                            squad = matchup.find('td', attrs={'class': 'td text team'})
                            winner = matchup.find('td', attrs={'class': 'td text team winner'})
                            loser = matchup.find('td', attrs={'class': 'td text team loser'})
                            win_percentage = matchup.find('td', attrs={'class': 'td number chance'}).get_text().strip()
                            
                            if squad:
                                team = squad.get_text().strip()
                                result = ''
                            elif winner:
                                team = winner.get_text().strip()
                                result = 'w'
                            elif loser:
                                team = loser.get_text().strip()
                                result = 'l'
                            else:
                                team = ''
                                result = ''
                            if num_teams ==  0:
                                df = df.append(pd.Series([team, win_percentage, date, result], 
                                               index=['team', 'win%', 'date', 'result']), ignore_index=True)
                            else:
                                df = df.append(pd.Series([team, win_percentage, '', result], 
                                               index=['team', 'win%', 'date', 'result']), ignore_index=True)
                                
                            

#div = soup.findAll('section', attrs={'class': 'week'}).get_text()
#div2 = soup.select('h3.days')#.week-group.week.days.daygames_wrap.game.game-bod')

#df.to_csv('z:\\Python Projects\\Score Predictor.csv', mode = 'a')
from datetime import datetime
datetime.strptime('Oct', '%b')
df.set_index('date')

In [ ]:
names = ['Open', 'odds','Westgate','MGM Mirage', 'betMGM', 
         'William Hill', 'CG Technology', 'Circa Sports','Stations']
book = pd.DataFrame(columns=names)
output_file = 'z:\python_projects\aaa.exe'
temp_away_list = []
temp_home_list = []
teams = []
URL = 'http://www.vegasinsider.com/nfl/odds/las-vegas/money/'
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
source = requests.get(URL, headers=headers)
soup = BeautifulSoup(source.content, 'html.parser')

for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
    for games in gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'}):
        for team_names in games.findAll('td', attrs={'class': 'viCellBg1 cellTextNorm cellBorderL1'}):
            for team_name in team_names.findAll('a', attrs={'class': 'tabletext'}):
                teams.append(team_name.get_text())

        for team_names in games.findAll('td', attrs={'class': 'viCellBg2 cellTextNorm cellBorderL1'}):
            for team_name in team_names.findAll('a', attrs={'class': 'tabletext'}):
                teams.append(team_name.get_text())

        for num, spread in enumerate(games.findAll('td', attrs={'class': ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap',
                                                                          'viCellBg1 cellTextHot cellBorderL1 center_text nowrap']})):           
            spread_text = spread.get_text().strip()
            
            if spread_text == '' or spread_text == 'XXXX':
                away_spread = np.nan
                home_spread = np.nan
            elif(spread_text[4] == '+' or spread_text[4] == '-'):
                away_spread = spread_text[0:4]
                home_spread = spread_text[4:]
            else:
                away_spread = spread_text[0:5]
                home_spread = spread_text[5:]

            temp_away_list.append(away_spread)
            temp_home_list.append(home_spread)
            if len(temp_home_list) == 9:
                book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                temp_away_list = []
                temp_home_list = []
                    
        for num, spread in enumerate(games.findAll('td', attrs={'class': ['viCellBg2 cellTextNorm cellBorderL1 center_text nowrap',
                                                                          'viCellBg2 cellTextHot cellBorderL1 center_text nowrap']})):           
            spread_text = spread.get_text().strip()

            if spread_text == '' or spread_text == 'XXXX':
                away_spread = np.nan
                home_spread = np.nan
            elif(spread_text[4] == '+' or spread_text[4] == '-'):
                away_spread = spread_text[0:4]
                home_spread = spread_text[4:]
            else:
                away_spread = spread_text[0:5]
                home_spread = spread_text[5:]
            temp_away_list.append(away_spread)
            temp_home_list.append(home_spread)

            if len(temp_home_list) == 9:
                book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                temp_away_list = []
                temp_home_list = []

                    #print(away_spread)
                    #print(home_spread)
book['team'] = teams[0:len(book.index)]
book.set_index('team', inplace=True)

book.dropna(how='all', inplace=True)
book = book.reindex(index=df.team)
book['team'] = book.index
book.replace(np.nan, '', inplace=True)
odds = book[['odds', 'team']]
book

In [ ]:
spreadsheet = pd.merge(odds, df)
spreadsheet = spreadsheet[['date', 'team', 'win%', 'result', 'odds']]
spreadsheet.set_index('date')

In [ ]:
spreadsheet = pd.merge(odds, df)
spreadsheet = spreadsheet[['date', 'team', 'win%', 'result', 'odds']]
spreadsheet.set_index('date')
spreadsheet['implied%'] = spreadsheet.apply(lambda row: implied_probability(row),
                                            axis=1)
spreadsheet['pick'] = spreadsheet.apply(lambda row: pick(row), axis=1)
spreadsheet['w/l'] = spreadsheet.apply(lambda row: win_loss(row), axis=1)
spreadsheet = spreadsheet[['date', 'team', 'win%', 'odds', 'implied%', 
                           'pick', 'result', 'w/l']]
spreadsheet

In [ ]:
def implied_probability(row):
    """Uses odds to determine implied probability
    Args:
        row: row of data from dataframe
    Returns:
        implied win probability if it exists
    """
    if row['odds'] == '':
        return('')
    elif row['odds'][0] == '+':
        return(100.0/(100+int(row['odds'][1:])))
    elif row['odds'][0] == '-':
        return(int(row['odds'][1:])/(100.0+int(row['odds'][1:])))
    else:
        return('')

In [ ]:
def pick(row):
    """Uses win% and odds to determine what team to pick
    Args:
        row: row of data from dataframe
    Returns:
        pick if there is one
    """
    if row['implied%'] == '':
        return('')
    elif (float(row['win%'][:-1])/100.0 > row['implied%']):
        return(row.team)
    else:
        return('')
    

In [ ]:
def win_loss(row):
    """Determines amount won lost
    Args:
        row: row of data from dataframe
    Returns:
        =amount won/lost
    """
    if row['pick'] == '':
        return('')
    else:
        if row['result'] == 'w':
            if row['odds'][0] == '+':
                return(row['odds'][1:])
            else:
                return(100)
        elif row['result'] == 'l':
            if row['odds'][0] == '+':
                return(-100)
            else:
                return(row['odds'][1:]) 
        else:
            return('')
        